# Génération des Prédictions pour 2027

**Objectif :** Ce notebook est la dernière étape du pipeline. Il utilise le modèle entraîné pour prédire les résultats des élections de 2027 et prépare les données pour l'analyse en Business Intelligence (BI).

1.  **Chargement du modèle** et des transformateurs sauvegardés.
2.  **Récupération des données de 2024** depuis PostgreSQL pour servir de base à la prédiction.
3.  **Génération des prédictions** pour l'année 2027.
4.  **Consolidation** des résultats historiques avec les nouvelles prédictions.
5.  **Sauvegarde** de la table finale (`election_results_for_bi`) dans PostgreSQL.

## 1. Imports et Configuration

In [1]:
import pandas as pd
import joblib
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
print("Librairies importées et variables d'environnement chargées.")

Librairies importées et variables d'environnement chargées.


## 2. Connexion à PostgreSQL et Définition des Chemins

In [2]:
MODELS_DIR = '../models'
ARTIFACTS_DIR = '../database'
PREPROCESSOR_PATH = os.path.join(ARTIFACTS_DIR, 'preprocessor_X.joblib')
LABEL_ENCODER_PATH = os.path.join(ARTIFACTS_DIR, 'label_encoder_y.joblib')

BEST_MODEL_FILENAME = 'random_forest_predictor.joblib'
MODEL_PATH = os.path.join(MODELS_DIR, BEST_MODEL_FILENAME)

pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host = os.getenv("PG_HOST")
pg_port = os.getenv("PG_PORT")
pg_dbname = os.getenv("PG_DBNAME")

if not all([pg_user, pg_password, pg_host, pg_port, pg_dbname]):
    raise ValueError("Fichier .env incomplet. Vérifiez les variables PG_USER, PG_PASSWORD, etc.")

db_url = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}"
engine = create_engine(db_url)

print(f"Connexion à la base '{pg_dbname}' établie.")

Connexion à la base 'ELECTIONS' établie.


## 3. Chargement du Modèle et des Transformateurs

In [3]:
print("Chargement du modèle et des transformateurs...")
try:
    model = joblib.load(MODEL_PATH)
    preprocessor_X = joblib.load(PREPROCESSOR_PATH)
    label_encoder_y = joblib.load(LABEL_ENCODER_PATH)
    print(f"✅ Modèle '{BEST_MODEL_FILENAME}' et transformateurs chargés.")
except FileNotFoundError as e:
    print(f"❌ Erreur : Fichier non trouvé. {e}")
    print("Veuillez d'abord exécuter le notebook 'model_training.ipynb'.")
except Exception as e:
    print(f"Une erreur est survenue : {e}")

Chargement du modèle et des transformateurs...


✅ Modèle 'random_forest_predictor.joblib' et transformateurs chargés.


## 4. Préparation des Données pour la Prédiction

In [4]:
print("Chargement des données de 2024 comme base pour la prédiction de 2027...")
try:
    # CORRECTION : Utiliser les noms de colonnes exacts (majuscules, avec guillemets) de la vue 
    # et utiliser des alias (AS) pour obtenir un DataFrame avec des noms en minuscules.
    query = """
    SELECT 
        "DEPARTMENT_CODE"   AS department_code,
        "NUMBER_OF_VICTIMS" AS number_of_victims, 
        "IMMIGRATION_RATE"  AS immigration_rate, 
        "POVERTY_RATE"      AS poverty_rate, 
        "UNEMPLOYMENT_RATE" AS unemployment_rate
    FROM elections_all
    WHERE "YEAR" = 2024
    """
    features_to_predict = pd.read_sql(query, engine)
    print(f"✅ {len(features_to_predict)} lignes chargées pour la prédiction.")
    
    if features_to_predict.isnull().sum().sum() > 0:
        raise ValueError("Des valeurs NaN ont été trouvées dans les données de prédiction.")
    print("✅ Aucune valeur NaN détectée.")
except Exception as e:
    print(f"❌ Erreur lors du chargement des données de prédiction : {e}")

Chargement des données de 2024 comme base pour la prédiction de 2027...
✅ 94 lignes chargées pour la prédiction.
✅ Aucune valeur NaN détectée.


## 5. Génération des Prédictions pour 2027

In [5]:
print("\nPré-traitement des données et génération des prédictions pour 2027...")

department_codes = features_to_predict['department_code'].copy()

X_processed = preprocessor_X.transform(features_to_predict)
predictions_encoded = model.predict(X_processed)
predictions_decoded = label_encoder_y.inverse_transform(predictions_encoded)

predictions_2027_df = pd.DataFrame({
    'year': 2027,
    'department_code': department_codes,
    'winner': predictions_decoded
})

print("✅ Prédictions pour 2027 générées.")
display(predictions_2027_df.head())


Pré-traitement des données et génération des prédictions pour 2027...
✅ Prédictions pour 2027 générées.


,year,department_code,winner
0,2027,01,GAUCHE
1,2027,02,E.DROITE
2,2027,03,GAUCHE
3,2027,04,GAUCHE
4,2027,05,CENTRE


## 6. Consolidation avec les Données Historiques

In [6]:
print("\nChargement des résultats historiques...")
try:
    # CORRECTION : Même chose pour la requête des données historiques.
    query = """
    SELECT DISTINCT 
        "YEAR"            AS year, 
        "DEPARTMENT_CODE" AS department_code, 
        "WINNER"          AS winner
    FROM elections_all
    WHERE "WINNER" IS NOT NULL
    """
    historical_results_df = pd.read_sql(query, engine)
    print(f"✅ {len(historical_results_df)} lignes de résultats historiques chargées.")

    final_bi_table = pd.concat([historical_results_df, predictions_2027_df], ignore_index=True)
    print(f"La table finale pour la BI contient {len(final_bi_table)} lignes, de {final_bi_table['year'].min()} à {final_bi_table['year'].max()}.")
except Exception as e:
    print(f"❌ Erreur lors du chargement des données historiques : {e}")


Chargement des résultats historiques...
✅ 281 lignes de résultats historiques chargées.
La table finale pour la BI contient 375 lignes, de 2017 à 2027.


## 7. Sauvegarde de la Table Finale pour la BI

In [7]:
print("\nSauvegarde de la table finale dans PostgreSQL...")
try:
    final_bi_table.to_sql(
        'election_results_for_bi', 
        engine, 
        if_exists='replace', 
        index=False
    )
    print("✅ Table 'election_results_for_bi' sauvegardée avec succès.")
    print("Vous pouvez maintenant la connecter à votre outil de BI.")
except Exception as e:
    print(f"❌ Erreur lors de la sauvegarde de la table finale : {e}")


Sauvegarde de la table finale dans PostgreSQL...
✅ Table 'election_results_for_bi' sauvegardée avec succès.
Vous pouvez maintenant la connecter à votre outil de BI.


## 8. Fermeture de la Connexion

In [8]:
engine.dispose()
print("\nConnexion à la base de données fermée.")


Connexion à la base de données fermée.
